In [1]:
!pip install transformers
import pandas as pd
import numpy as np
from tqdm.auto import tqdm  # progress bar
import tensorflow as tf
from datetime import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Tokenize text & create dataset

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # ft -> maps word to numerical value in berts vocab

In [4]:

def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

def create_dataset(df, label_name = 'ArgumentLevel', label_number = 5, batch_size = 16, max_length = 256):

    # Tokenize text
    X_input_ids = np.zeros((len(df), max_length))
    X_attn_masks = np.zeros((len(df), max_length))

    for i, text in tqdm(enumerate(df['Content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = max_length,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
            return_tensors = 'tf'
        )
        
        X_input_ids[i,:] = tokenized_text.input_ids
        X_attn_masks[i,:] = tokenized_text.attention_mask
    

    # Generate labels - One-hot encoding
    labels = np.zeros((len(df), label_number)) 
    labels[np.arange(len(df)), df[label_name].values] = 1


    # Create dataset object
    dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

    # Map dataset
    dataset = dataset.map(DatasetMapFunction)

    # shuffle data
    dataset = dataset.shuffle(400,reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True) # drop remainder 179/16 = 11...3, drop 3 data

    return dataset

Split training and validation data 

In [5]:
def split_dataset(dataset, p = 0.5, batch_size = 16):
    # p - training data size %
    train_size = int((len(df)//batch_size)*p)
    
    train_dataset = dataset.take(train_size) # take the first 8 batches
    val_dataset = dataset.skip(train_size) # take the last 3 batches

    return train_dataset, val_dataset

Create model

In [6]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [7]:
def create_model(label_number = 5, max_length = 256):

    # input layer
    input_ids = tf.keras.layers.Input(shape = (max_length), name = 'input_ids', dtype = 'int32')
    # attention layer
    attention_masks = tf.keras.layers.Input(shape = (max_length), name = 'attention_mask', dtype = 'int32')

    # bert layer
    bert_layer = bert_model.bert(input_ids, attention_mask=attention_masks)[1] 

    # intermediate layer
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name = 'intermediate_layer')(bert_layer)

    # output layer - 5 layers because 5 label classes
    output_layer = tf.keras.layers.Dense(label_number, activation='softmax', name = 'output_layer')(intermediate_layer)

    # Create model object
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)



    # Add optimizer & Loss function & Accuracy metrics
    optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracry')

    model.compile(optimizer = optim, loss = loss_func, metrics = [acc])

    return model


========= 分界线 ==========
==========================

In [8]:
year = 'Y1Y2'
label_name = 'ArgumentLevel'
label_number = 5

run_num = 1
# e = 4 # epochs = 4 for previous runs

In [9]:
folderpath = '/content/drive/MyDrive/Imperial/nlp-physicseducation/outputs'
outputpath = '/content/drive/MyDrive/Imperial/BERT-results/20230309-epochs'
filepath = '/sections/labels_cleaned_{year}.csv'.format(year = year)

Load data

In [10]:
df = pd.read_csv(folderpath+filepath)
# df.head()
df['ArgumentLevel'] = df['ArgumentLevel'].replace({'superficial': 0, 'extended': 1, 'deep': 2, 'expert': 3, 'prediction': 4})
df['ReasoningLevel'] = df['ReasoningLevel'].replace({'bal': 0, 'the': 1, 'exp': 2, 'none': 3})
# df['ReasoningLevel'].value_counts()

/Users/jiayangzhang/Library/CloudStorage/GoogleDrive-jiayang.zhang@icloud.com/My Drive/Imperial/nlp-physicseducation/outputs/sections

In [11]:
s = 8
epochs = [1,2,3,4,5,6,7,8,9,10]
for e in epochs:
# train_ratios = [2,3,4,5,6,7,8,9] 
# for s in train_ratios: # training ratio if p=5, train ratio = 0.5
    for i in range(run_num):
        # about dataset
        dataset = create_dataset(df, label_name = label_name, label_number = label_number, batch_size = 16, max_length=512) # create dataset
        train_dataset, val_dataset = split_dataset(dataset, p = (0.1*s), batch_size = 16) # split training and validation data 


        # about model
        model = create_model(label_number = label_number, max_length = 512)   # Create model


        # about train and validate
        history = model.fit(     # Train & Validate model (Fine-tuning)
            train_dataset,
            validation_data = val_dataset,
            epochs = e
        )


        history_saveto = outputpath + '/{year}/{label_name}/trainsize0{size}/{time}_{year}_{label_name}_{size}trainsize_{e}epochs_16batchsize_1e-5lr.npy'.format(
            time = datetime.now().strftime("%H%M%S"), 
            year = year, 
            label_name = label_name, 
            size = s,
            e = e
            )
        np.save(history_saveto, history.history) # save history


        # clear model after training
        tf.keras.backend.clear_session()
        import gc
        gc.collect()
        del model

0it [00:00, ?it/s]

8/8 [==============================] - 41s 579ms/step - loss: 1.5302 - accuracry: 0.3203 - val_loss: 1.2806 - val_accuracry: 0.2708


0it [00:00, ?it/s]

Epoch 1/2
8/8 [==============================] - 39s 616ms/step - loss: 1.4916 - accuracry: 0.3359 - val_loss: 1.4235 - val_accuracry: 0.3750
Epoch 2/2
8/8 [==============================] - 3s 322ms/step - loss: 1.4194 - accuracry: 0.3359 - val_loss: 1.3479 - val_accuracry: 0.4167


0it [00:00, ?it/s]

...

Epoch 1/8
8/8 [==============================] - 40s 720ms/step - loss: 1.4399 - accuracry: 0.3906 - val_loss: 1.1971 - val_accuracry: 0.6042
Epoch 2/8
8/8 [==============================] - 2s 286ms/step - loss: 1.1064 - accuracry: 0.6016 - val_loss: 1.0725 - val_accuracry: 0.5625
Epoch 3/8
8/8 [==============================] - 2s 279ms/step - loss: 1.0616 - accuracry: 0.5703 - val_loss: 1.0178 - val_accuracry: 0.5417
Epoch 4/8
8/8 [==============================] - 2s 264ms/step - loss: 1.0589 - accuracry: 0.5391 - val_loss: 1.0006 - val_accuracry: 0.6042
Epoch 5/8
8/8 [==============================] - 2s 287ms/step - loss: 0.9717 - accuracry: 0.5938 - val_loss: 1.0784 - val_accuracry: 0.6250
Epoch 6/8
8/8 [==============================] - 2s 296ms/step - loss: 0.9611 - accuracry: 0.5781 - val_loss: 0.9634 - val_accuracry: 0.6875
Epoch 7/8
8/8 [==============================] - 2s 265ms/step - loss: 0.9742 - accuracry: 0.5859 - val_loss: 0.9276 - val_accuracry: 0.6042
Epoch 8/8
8/

0it [00:00, ?it/s]

Epoch 1/9
8/8 [==============================] - 40s 539ms/step - loss: 1.4167 - accuracry: 0.3281 - val_loss: 1.1682 - val_accuracry: 0.5625
Epoch 2/9
8/8 [==============================] - 2s 291ms/step - loss: 1.1037 - accuracry: 0.5469 - val_loss: 0.8297 - val_accuracry: 0.6875
Epoch 3/9
8/8 [==============================] - 2s 292ms/step - loss: 0.9874 - accuracry: 0.5703 - val_loss: 0.8980 - val_accuracry: 0.6667
Epoch 4/9
8/8 [==============================] - 2s 271ms/step - loss: 0.9818 - accuracry: 0.6484 - val_loss: 0.8545 - val_accuracry: 0.6250
Epoch 5/9
8/8 [==============================] - 2s 274ms/step - loss: 0.8758 - accuracry: 0.5859 - val_loss: 0.8933 - val_accuracry: 0.7083
Epoch 6/9
8/8 [==============================] - 2s 271ms/step - loss: 0.8978 - accuracry: 0.6094 - val_loss: 0.9359 - val_accuracry: 0.5417
Epoch 7/9
8/8 [==============================] - 2s 262ms/step - loss: 0.8514 - accuracry: 0.6406 - val_loss: 0.8320 - val_accuracry: 0.7083
Epoch 8/9
8/

0it [00:00, ?it/s]

Epoch 1/10
8/8 [==============================] - 40s 546ms/step - loss: 1.5443 - accuracry: 0.3516 - val_loss: 1.3880 - val_accuracry: 0.4583
Epoch 2/10
8/8 [==============================] - 2s 301ms/step - loss: 1.2295 - accuracry: 0.5391 - val_loss: 1.0312 - val_accuracry: 0.6667
Epoch 3/10
8/8 [==============================] - 2s 283ms/step - loss: 1.0722 - accuracry: 0.5938 - val_loss: 0.9758 - val_accuracry: 0.6250
Epoch 4/10
8/8 [==============================] - 2s 279ms/step - loss: 0.9103 - accuracry: 0.6484 - val_loss: 0.8619 - val_accuracry: 0.7500
Epoch 5/10
8/8 [==============================] - 2s 270ms/step - loss: 0.9435 - accuracry: 0.6406 - val_loss: 0.8237 - val_accuracry: 0.7500
Epoch 6/10
8/8 [==============================] - 2s 289ms/step - loss: 0.7450 - accuracry: 0.7891 - val_loss: 0.8096 - val_accuracry: 0.7083
Epoch 7/10
8/8 [==============================] - 2s 282ms/step - loss: 0.6567 - accuracry: 0.7891 - val_loss: 0.4008 - val_accuracry: 0.9167
Epoch

Save model

In [12]:
model_saveto = './}_{}_trainsize{}'
model.save('Y1Y2_argument_model')

NameError: ignored